In [187]:
class Node:
    
    def __init__(self, Node1 = None, Node2 = None):
        """
        A binary tree is either a leaf or a node with two subtrees.
        
        INPUT:
            
            - children, either None (for a leaf), or a list of size excatly 2 
            of either two binary trees or 2 objects that can be made into binary trees
        """
        self._isleaf = Node1 is None and Node2 is None
        if not self._isleaf:
            if Node1 is None or Node2 is None != 2:
                raise ValueError("A binary tree needs exactly two children")
            self._children = tuple(c if isinstance(c,Node) else Node(c) for c in [Node1, Node2])
        self._size = None
        
    def __repr__(self):
        if self.is_leaf():
            return "Leaf"
        else:
            return "Node" + str(self._children)
        
    def __eq__(self, other):
        """
        Return true if other represents the same binary tree as self
        """
        if not isinstance(other, BinaryTree):
            return False
        if self.is_leaf():
            return other.is_leaf()
        if other.is_leaf():
            return False
        return self.left() == other.left() and self.right() == other.right()
    
    
    def left(self):
        """
        Return the left subtree of self
        """
        return self._children[0]
    
    def right(self):
        """
        Return the right subtree of self
        """
        return self._children[1]
    
    def is_leaf(self):
        """
        Return true is self is a leaf
        """
        return self._isleaf
    
    def _compute_size(self):
        """
        Recursively computes the size of self
        """
        if self.is_leaf():
            self._size = 0
        else:
            self._size = self.left().size() + self.right().size() + 1
    
    def size(self):
        """
        Return the number of non leaf nodes in the binary tree
        """
        if self._size is None:
            self._compute_size()
        return self._size
    
    def height(self):
        if self._isleaf:
            return 0
        else:
            aux_left = self.left()
            height_left = aux_left.height()
            aux_right = self.right()
            height_right = aux_right.height()
            return 1+max(height_left, height_right)
        
Leaf = Node()
        

In [188]:
tr = Node(Leaf, Node(Leaf, Leaf))
tr

Node(Leaf, Node(Leaf, Leaf))

In [279]:
class AbstractRule:
    
    def _set_grammar(self, gram):
        self._grammar = gram
        
class ConstructorRule(AbstractRule):
    
    def __init__(self, fst, snd):
        self._parameters = (fst, snd)
        self._valuation = float("inf")
    
    def valuation(self):
        return self._valuation
        
    def _update_valuation(self):
        self._valuation = self._calc_valuation()
        
class UnionRule(ConstructorRule):
    
    def __init__(self, fst, snd):
        ConstructorRule.__init__(self, fst, snd)
        
    def _calc_valuation(self):
        return min([self._grammar[parameter].valuation() for parameter in self._parameters])
    
    def count(self, n):
        return sum([self._grammar[parameter].count(n) for parameter in self._parameters])
    
    def list(self, n):
        answer = []
        for parameter in self._parameters:
            answer += self._grammar[parameter].list(n)
        return answer
    
    def unrank(self, n, i):
        if i >= self.count(n):
            raise ValueError
        else:
            C_a = self._grammar[self._parameters[0]].count(n)
            if i < C_a:
                return self._grammar[self._parameters[0]].unrank(n, i)
            else:
                return self._grammar[self._parameters[1]].unrank(n, i - C_a)
    
class ProductRule(ConstructorRule):
    
    def __init__(self, fst, snd, cons):
        ConstructorRule.__init__(self, fst, snd)
        self._constructor = cons
        
    def _calc_valuation(self):
        return sum([self._grammar[parameter].valuation() for parameter in self._parameters])
    
    def count(self, n):
        total_sum = 0
        vals = [self._grammar[parameter].valuation() for parameter in self._parameters]
        for k in range(vals[0], n+1):
            l = n - k
            if l >= vals[1]:
                aux = [self._grammar[self._parameters[0]].count(k), self._grammar[self._parameters[1]].count(l)]
                total_sum += aux[0]*aux[1]
        return total_sum
    
    def list(self, n):
        answer = []
        vals = [self._grammar[parameter].valuation() for parameter in self._parameters]
        for k in range(vals[0], n+1):
            l = n - k
            if l >= vals[1]:
                auxans = (self._grammar[self._parameters[0]].list(k), self._grammar[self._parameters[1]].list(l))
                for elem0 in auxans[0]:
                    for elem1 in auxans[1]:
                        answer.append(self._constructor((elem0, elem1)))
        return answer
    
    def unrank(self, n, i):
        if i >= self.count(n):
            raise ValueError
        else:
            auxi = i
            auxprod = []
            for index in range(n+1):
                templist = [self._grammar[self._parameters[0]].count(index), self._grammar[self._parameters[0]].count(n-index)]
                auxprod.append(reduce(lambda x, y: x*y, templist))
                if auxi - auxprod[-1] < 0:
                    newi = index
                    Node1 = self._grammar[self._parameters[0]]
                    Node2 = self._grammar[self._parameters[1]]
                    Node1rang = auxi%Node1.count(newi)
                    Node2rang = auxi/Node1.count(newi)
                    return self._constructor((Node1.unrank(newi, Node1rang), Node2.unrank(n - newi, Node2rang)))
                else:
                    auxi -= auxprod[-1]
                    
class ConstantRule(AbstractRule):
    
    def __init__(self, object):
        self._object = object
        
class SingletonRule(ConstantRule):
    
    def __init__(self, obj):
        ConstantRule.__init__(self, obj)
        
    def valuation(self):
        return 1
    
    def count(self, n):
        if n == 1:
            return 1
        else:
            return 0
        
    def list(self, n):
        if n == 1:
            return [self._object]
        else:
            return []
        
    def unrank(self, n, i):
        if i >= self.count(n):
            raise ValueError
        else:
            return self._object
    
class EpsilonRule(ConstantRule):
    
    def __init__(self, obj):
        ConstantRule.__init__(self, obj)
        
    def valuation(self):
        return 0
    
    def count(self, n):
        if n == 0:
            return 1
        else:
            return 0
        
    def list(self, n):
        if n == 0:
            return [self._object]
        else:
            return []
        
    def unrank(self, n, i):
        if i >= self.count(n):
            raise ValueError
        else:
            return self._object
        
def init_grammar(grammar):
    for rule_key in grammar:
        grammar[rule_key]._set_grammar(grammar)
    valuations = [float("inf")]*len(grammar)
    newvaluations = [grammar[key].valuation() for key in grammar]
    while newvaluations != valuations:
        valuations = newvaluations
        for rule_key in grammar:
            if isinstance(grammar[rule_key], ConstructorRule):
                grammar[rule_key]._update_valuation()
        newvaluations = [grammar[key].valuation() for key in grammar]
    return

In [280]:
treeGram = {"Tree" : UnionRule("Node", "Leaf"),
"Node" : ProductRule("Tree", "Tree",
lambda (a, b) : Node(a, b)),
"Leaf" : SingletonRule(Leaf)}
init_grammar(treeGram)

In [281]:
k = 4
for rule in treeGram:
    print rule, treeGram[rule].valuation(), treeGram[rule].count(k)
for tree in treeGram["Tree"].list(k): print tree

Node 2 5
Leaf 1 0
Tree 1 5
Node(Leaf, Node(Leaf, Node(Leaf, Leaf)))
Node(Leaf, Node(Node(Leaf, Leaf), Leaf))
Node(Node(Leaf, Leaf), Node(Leaf, Leaf))
Node(Node(Leaf, Node(Leaf, Leaf)), Leaf)
Node(Node(Node(Leaf, Leaf), Leaf), Leaf)


In [283]:
print treeGram["Tree"].unrank(7, 73)
print treeGram["Tree"].unrank(4, 2)
print treeGram["Tree"].unrank(3, 1)

Node(Node(Node(Leaf, Leaf), Node(Leaf, Leaf)), Node(Node(Leaf, Leaf), Leaf))
Node(Node(Leaf, Leaf), Node(Leaf, Leaf))
Node(Node(Leaf, Leaf), Leaf)
